In [1]:
import sys
sys.path
sys.path.append('../../modules')


import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import logging

#utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)
""
from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation


setting CUDA_VISIBLE_DEVICES =  0,


## Load the configuration for training

In [2]:
config_filename = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'dataset', 'composite', 'caterpillar_upright.yaml')
config = utils.getDictFromYamlFilename(config_filename)

train_config_file = os.path.join(utils.getDenseCorrespondenceSourceDir(), 'config', 'dense_correspondence', 
                               'training', 'training.yaml')

train_config = utils.getDictFromYamlFilename(train_config_file)
dataset = SpartanDataset(config=config)

logging_dir = "trained_models/tutorials"
num_iterations = 3500
d = 3 # the descriptor dimension
name = "caterpillar_%d" %(d)
train_config["training"]["logging_dir_name"] = name
train_config["training"]["logging_dir"] = logging_dir
train_config["dense_correspondence_network"]["descriptor_dimension"] = d
train_config["training"]["num_iterations"] = num_iterations

TRAIN = True
EVALUATE = True

Using SpartanDataset:
   - in train mode
   - number of scenes 11
   - total images:     2851


## Train the network

This should take about ~12-15 minutes with a GTX 1080 Ti

In [12]:
import torch

In [23]:
class Test (torch.nn.Module):
    def __init__(self):
        super(Test, self).__init__()
        self.layer = torch.nn.Linear(1, 10)
    
    def forward (self, x):
        return self.layer(x)

In [24]:
t = Test()

In [30]:
t.to(torch.device('cuda'))

Test(
  (layer): Linear(in_features=1, out_features=10, bias=True)
)

In [32]:
t(torch.rand([20,1]).to(torch.device('cuda')))

RuntimeError: cublas runtime error : the GPU program failed to execute at /pytorch/aten/src/THC/THCBlas.cu:259

In [3]:
# All of the saved data for this network will be located in the
# code/data/pdc/trained_models/tutorials/caterpillar_3 folder

if TRAIN:
    print "training descriptor of dimension %d" %(d)
    train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
    train.run()
    print "finished training descriptor of dimension %d" %(d)


training descriptor of dimension 3
using SINGLE_OBJECT_WITHIN_SCENE
logging_dir: /home/michelism/data/pdc/trained_models/tutorials/caterpillar_3


Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/michelism/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100.0%


RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED

## Evaluate the network quantitatively

This should take ~5 minutes.

In [ ]:
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)

if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    num_image_pairs = 100
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)      

See `evaluation_quantitative_tutorial.ipynb` for a better place to display the plots.